In [14]:
import random
k = 2
total_time = 0

In [15]:
#-------------------------------------------------------------
# Public parameter generator input = degree k
#-------------------------------------------------------------
def parameter_gen(k):
    flag = True
    while flag :
        N = next_prime(randrange(2**k, 2**(k+1)))
        p = next_prime(randrange(2**k, 2**(k+1)))
        d = randrange(1, N)
        q = next_prime(randrange((6*d + 1)*p, (6*d + 1)*(p*2^5)))
        if gcd(p,q) == 1 :
            if gcd(N,q) == 1 :
                if q > (6*d + 1) * p :
                    flag = False
    return (N,p,q,d)
#-------------------------------------------------------------
t = cputime()
(N,p,q,d) = parameter_gen(k)
#(N,p,q,d) = (7,3,41,2)
ti = cputime(t)
total_time += ti
print ("Cpu time = " + str(ti))
print ("(N,p,q,d) = " + str((N,p,q,d)))
print(n(N*log(q,2)))

Cpu time = 0.000299
(N,p,q,d) = (11, 11, 307, 4)
90.8830432990720


In [16]:
#-------------------------------------------------------------
# Ring Mapping
#-------------------------------------------------------------
# ZZ[x] --> R=ZZ[x]/(x^N-1)
def map_to_R(a):
    R.<x> = PolynomialRing(ZZ)
    idR=R.ideal(x^N-1)
    QuoR.<x>=R.quotient_ring(idR)
    return QuoR(a)
#-------------------------------------------------------------
# ZZ[x] --> Rp=ZZp[x]/(x^N-1)

def map_to_Rp(a):
    Rp=IntegerModRing(p)
    Rp.<x>=PolynomialRing(Rp)
    idRp=Rp.ideal(x^N-1)
    QuoRp.<x>=Rp.quotient_ring(idRp)
    return QuoRp(a)
#-------------------------------------------------------------
# ZZ[x] --> Rq=ZZq[x]/(x^N-1)

def map_to_Rq(a):
    Rq=IntegerModRing(q)
    Rq.<x>=PolynomialRing(Rq)
    idRq=Rq.ideal(x^N-1)
    QuoRq.<x>=Rq.quotient_ring(idRq)
    return QuoRq(a)
#-------------------------------------------------------------
# Tripolynomial
#-------------------------------------------------------------
def tri_poly(d_1, d_2) :
    s = [1 for j in range(d_1 - 1)]
    s = s + [-1 for j in range(d_2)]
    s = s + [0 for j in range(N - d_2 - d_1)]
    random.shuffle(s)
    s.append(1)
    return map_to_R(s)
#-------------------------------------------------------------
# Fq = f^(-1) in Rq
#-------------------------------------------------------------
def fq_inv(f) :
    return map_to_Rq(f)^(-1)
#-------------------------------------------------------------
# Fp = f^(-1) in Rp
#-------------------------------------------------------------
def fp_inv(f) :
    return map_to_Rp(f)^(-1)
#-------------------------------------------------------------
def find_h(F_q, g) :
    return map_to_Rq(g)*F_q


In [17]:
#-------------------------------------------------------------
# Key generated by Alice
#-------------------------------------------------------------
#(N,p,q,d) = (7,3,41,2)
#-------------------------------------------------------------
t = cputime()
f = tri_poly(d+1, d)
g = tri_poly(d, d)
F_q = fq_inv(f)
F_p = fp_inv(f)
h = find_h(F_q, g)
ti = cputime(t)
total_time += ti
print ("Cpu time = " + str(ti))
print ("Alice's private key (f,F_p): ")
show((f,F_p))
print ("Alice's public key (h): ")
show(h)
#latex(f)
#latex(F_p)
#latex(h)

Cpu time = 0.066072
Alice's private key (f,F_p): 


(x^10 - x^9 + x^8 + x^7 + x^6 - x^5 - x^4 + x^2 - 1,
 5*x^10 + 3*x^9 + 2*x^8 + 7*x^7 + 9*x^6 + 9*x^5 + 6*x^4 + 7*x^3 + 9*x^2 + 10)

Alice's public key (h): 


156*x^10 + 10*x^9 + 33*x^8 + 107*x^7 + 175*x^6 + 170*x^5 + 126*x^4 + 160*x^3 + 33*x^2 + 54*x + 204

In [19]:
#-------------------------------------------------------------
# Encryption Function
# m in R
#-------------------------------------------------------------
def encrypt(m, h, r):
    m=map_to_R(m)
    r=map_to_R(r)
    return map_to_Rq((p*r*h)+m)
#-------------------------------------------------------------
# Bob encrypt message
#-------------------------------------------------------------
m = random.sample(range(0, p), N)
m = map_to_R(m)
#m=map_to_R(-x^5+x^3+x^2-x+1)
#m = tri_poly(3,2)
#-------------------------------------------------------------
# Ephimeral key
#-------------------------------------------------------------
#r = x^6 - x^5 + x-1
r = tri_poly(d,d)
#-------------------------------------------------------------
t = cputime()
e = encrypt(m, h, r)
ti = cputime(t)
total_time += ti
print ("Cpu time = " + str(ti))
print ("Plant text : ")
show(m)
#latex(m)
print ("Cipher text: ")
show(e)
#latex(e)

Cpu time = 0.005233
Plant text : 


4*x^10 + 5*x^9 + 8*x^8 + 2*x^7 + 10*x^6 + 6*x^5 + x^4 + 3*x^3 + 9*x^2 + 7*x

Cipher text: 


249*x^10 + 210*x^9 + 137*x^8 + 173*x^7 + 169*x^6 + 222*x^5 + 96*x^4 + 278*x^3 + 15*x^2 + 98*x + 250

In [20]:
#-------------------------------------------------------------
# Center lift function
#-------------------------------------------------------------
def center_lift(a, t) :
    lst_a = list(a)
    for i in range(len(lst_a)) :
        if lst_a[i] <= -t/2 :
            lst_a[i] += t
        elif lst_a[i] > t/2 :
            lst_a[i] -= t
    return map_to_R(lst_a)
#-------------------------------------------------------------
# Decrypt recieved cypher text e(x)
#-------------------------------------------------------------

def decrypt(f, F_p, e) :
    f = map_to_R(f)
    a = map_to_Rq(f*e)
    a = map_to_R(a)
    #a = ((f * e) % id) % q
    a = center_lift(a, q)
    mhat = map_to_Rp(F_p*a)
    mhat = map_to_R(mhat)
    mhat = center_lift(mhat, p)
    return mhat
#-------------------------------------------------------------

In [21]:
#--------------------------------------------------------
# Alice decrypt cypher text
#--------------------------------------------------------
t = cputime()
mhat_lift = decrypt(f, F_p, e)
ti = cputime(t)
total_time += ti
print ("Cpu time = " + str(ti))
print ("Recovery message: ")
show(mhat_lift)
show(map_to_Rp(mhat_lift))
mhat_lift = map_to_Rp(mhat_lift)
print ("Plain text: ")
show(m)
#--------------------------------------------------------

Cpu time = 0.013886
Recovery message: 


4*x^10 + 5*x^9 - 3*x^8 + 2*x^7 - x^6 - 5*x^5 + x^4 + 3*x^3 - 2*x^2 - 4*x

4*x^10 + 5*x^9 + 8*x^8 + 2*x^7 + 10*x^6 + 6*x^5 + x^4 + 3*x^3 + 9*x^2 + 7*x

Plain text: 


4*x^10 + 5*x^9 + 8*x^8 + 2*x^7 + 10*x^6 + 6*x^5 + x^4 + 3*x^3 + 9*x^2 + 7*x

In [22]:
print("Total Time = " + str(total_time))

Total Time = 0.103071


In [23]:
print (mhat_lift == m)

True


In [24]:
#--------------------------------------------------------
# edit by the advisor
#--------------------------------------------------------
#(N,p,q,d) = (7,3,41,2)
(N,p,q,d) = parameter_gen(k)
print ("(N,p,q,d) = " + str((N,p,q,d)))
count = 0
for i in range(1000) :
    #f = x^6-x^4+x^3+x^2-1
    #g = x^6+x^4-x^2-x
    f = tri_poly(d+1, d)
    g = tri_poly(d, d)

    F_q = fq_inv(f)
    F_p = fp_inv(f)
    h = find_h(F_q, g)
    
    m = random.sample(range(0, p), N)
    m = map_to_Rp(m)
    #m = tri_poly(3,2)
    r = tri_poly(d,d)
    #r = x^6 - x^5 + x-1
    
    e = encrypt(m, h, r)
    
    mhat_lift = decrypt(f, F_p, e)
    mhat_lift = map_to_Rp(mhat_lift)
    if mhat_lift != m :
        count += 1
print ("Error = " + str(count))

(N,p,q,d) = (7, 5, 2767, 3)


ValueError: sample larger than population

In [28]:
import random
k = 2
total_time = 0
#--------------------------------------------------------
# Public parameter
#--------------------------------------------------------
print ("#--------------------------------------------------------\n# Public parameter\n#--------------------------------------------------------")
t = cputime()
(N,p,q,d) = parameter_gen(k)
#(N,p,q,d) = (7,3,41,2)
ti = cputime(t)
#total_time += ti
print ("Cpu time = " + str(ti))
print ("(N,p,q,d) = " + str((N,p,q,d)))
print("Key size = " + str(n(N*log(q,2))))
#--------------------------------------------------------
# Public key gen
#--------------------------------------------------------
print ("#--------------------------------------------------------\n# Public key gen\n#--------------------------------------------------------")
t = cputime()
f = tri_poly(d+1, d)
g = tri_poly(d, d)
F_q = fq_inv(f)
F_p = fp_inv(f)
h = find_h(F_q, g)
ti = cputime(t)
#total_time += ti
print ("Cpu time = " + str(ti))
print ("Alice's private key (f,F_p): ")
show((f,F_p))
print ("Alice's public key (h): ")
show(h)
#--------------------------------------------------------
# Encrypt
#--------------------------------------------------------
print ("#--------------------------------------------------------\n# Encrypt\n#--------------------------------------------------------")
m = random.sample(range(0, p), N)
m = map_to_R(m)
r = tri_poly(d,d)
t = cputime()
e = encrypt(m, h, r)
ti = cputime(t)
#total_time += ti
print ("Cpu time = " + str(ti))
print ("Plant text : ")
show(m)
print ("Cipher text: ")
show(e)
#--------------------------------------------------------
# Decrypt
#--------------------------------------------------------
print ("#--------------------------------------------------------\n# Decrypt\n#--------------------------------------------------------")
t = cputime()
mhat_lift = decrypt(f, F_p, e)
ti = cputime(t)
#total_time += ti
print ("Cpu time = " + str(ti))
print ("Recovery message: ")
show(mhat_lift)
show(map_to_Rp(mhat_lift))
mhat_lift = map_to_Rp(mhat_lift)
print ("Plain text: ")
show(m)
#--------------------------------------------------------
# Summary
#--------------------------------------------------------
print ("#--------------------------------------------------------\n# Summary\n#--------------------------------------------------------")
#print("Total Time = " + str(total_time))
print ("Recovery check = " + str(mhat_lift == m))

#--------------------------------------------------------
# Public parameter
#--------------------------------------------------------
Cpu time = 0.000336000000001
(N,p,q,d) = (7, 7, 1201, 2)
Key size = 71.6101430499394
#--------------------------------------------------------
# Public key gen
#--------------------------------------------------------
Cpu time = 0.034589
Alice's private key (f,F_p): 


(x^6 - x^5 + x^4 + x^3 - x, 2*x^5 + x^2 + 2*x + 3)

Alice's public key (h): 


510*x^6 + 892*x^5 + 361*x^4 + 32*x^3 + 957*x^2 + 1074*x + 978

#--------------------------------------------------------
# Encrypt
#--------------------------------------------------------
Cpu time = 0.018707
Plant text : 


...

-2*x^5 + 2*x^4 - 3*x^3 + x^2 - x + 3

5*x^5 + 2*x^4 + 4*x^3 + x^2 + 6*x + 3

Plain text: 


5*x^5 + 2*x^4 + 4*x^3 + x^2 + 6*x + 3

#--------------------------------------------------------
# Summary
#--------------------------------------------------------
Recovery check = True


In [86]:
#--------------------------------------------------------
# Public parameter
#--------------------------------------------------------
(N,p,q,d) = (7,3,41,2)
#--------------------------------------------------------
# Key generated by Alice
#--------------------------------------------------------
f = x^6-x^4+x^3+x^2-1
g = x^6+x^4-x^2-x
g = map_to_R(g)
F_q = fq_inv(f)
F_p = fp_inv(f)
h = find_h(F_q, g)
show(F_q)
show(g)
print "Alice's private key (f,F_p): "
show((f,F_p))
print "Alice's public key (h): "
show(h)
#--------------------------------------------------------
# Bob encrypt message
#--------------------------------------------------------
m = -x^5 + x^3 + x^2 - x + 1
#--------------------------------------------------------
# Ephimeral key
#--------------------------------------------------------
r= x^6 - x^5 + x-1
#--------------------------------------------------------
e = encrypt(m, h, r)
print "Plant text : "
show(m)
print "Cypher text : "
show(e)

8*x^6 + 26*x^5 + 31*x^4 + 21*x^3 + 40*x^2 + 2*x + 37

x^6 + x^4 - x^2 - x

Alice's private key (f,F_p): 


(x^6 - x^4 + x^3 + x^2 - 1, x^6 + 2*x^5 + x^3 + x^2 + x + 1)

Alice's public key (h): 


20*x^6 + 40*x^5 + 2*x^4 + 38*x^3 + 8*x^2 + 26*x + 30

Plant text : 


-x^5 + x^3 + x^2 - x + 1

Cypher text : 


31*x^6 + 19*x^5 + 4*x^4 + 2*x^3 + 40*x^2 + 3*x + 25

In [85]:
list_fq = list(F_q)
list_g = list(g)
for i in range(len(list_g)):
    list_g[i] = list_g[i]%q
ans = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in range(len(list_fq)):
    for j in range(len(list_g)):
        ans[i+j] += list_fq[i]*list_g[j]
#R.<x> = PolynomialRing(ZZ)
map_to_R(ans)

20*x^6 + 40*x^5 + 43*x^4 + 79*x^3 + 8*x^2 + 26*x + 30